In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

np.random.seed(1234)

In [3]:
print(torch.cuda.get_device_name(torch.cuda.current_device()))  # GPU name

# CUDA support
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

NVIDIA GeForce GTX 1650
Using device: cuda


In [ ]:
class DNN(nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        modules = []
        for i in range(len(layers) - 2):  # Exclude last layer for activation
            modules.append(nn.Linear(layers[i], layers[i+1]))
            modules.append(nn.Tanh())
        modules.append(nn.Linear(layers[-2], layers[-1]))  # Last layer (no activation)
        self.network = nn.Sequential(*modules)

        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)

    def forward(self, x):
        return self.network(x)

In [ ]:
class BC_net():
    def __init__(self, x, lb, rb, layers):

        self.x = x

        self.lb = lb
        self.rb = rb

        # DNN
        self.dnn = DNN(layers).to(device)

        self.optimizer_adam = torch.optim.Adam(self.dnn.parameters(), lr=0.01)

    def model_value(self, x):
        u = self.dnn(x)
        return u
    
    def boundary_cond(self, cond, u, u_x, m, m_x):
        bc_loss = 0

        match cond:
            case 'pinned':
                bc_loss += u**2 + m**2
            case 'fixed':
                bc_loss += u**2 + u_x**2
            case 'free':
                bc_loss += m**2 + m_x**2
            case 'roller':
                bc_loss += u_x**2 + m_x**2
        return bc_loss
    

    



In [ ]:
class PDE_net():
    def __init__(self, x, layers):
        self.x = x

        self.layers = layers
        